# Project de structures de données avancées
## Étudiant: Khernuf Valid
## Formation: M1 Informatique
---

Importation des bibliothèques nécessaires :
- pandas : pour manipuler des données sous forme tabulaire, comme des fichiers CSV.
- annoy : pour construire un index de recherche approximative de plus proches voisins.
- numpy : pour les opérations numériques et sur les tableaux.

Annoy est utilisé dans ce projet car il permet de réaliser des recherches rapides de voisins approximatifs dans de grands ensembles de données. Contrairement à d'autres méthodes, Annoy est particulièrement efficace pour les grandes bases de données en raison de sa faible consommation de mémoire et de son temps de recherche rapide, ce qui le rend adapté aux recommandations en temps réel.

In [1]:
import pandas as pd
from annoy import AnnoyIndex
import numpy as np

Lecture et traitement du fichier de données sur les interactions utilisateur-film :
- Définir le chemin vers le fichier CSV.
- Charger le fichier dans un DataFrame avec les colonnes "user_id" et "film_id".
- Limiter la lecture à 500 000 lignes afin de réduire la charge sur la mémoire vive (RAM).
- Supprimer les lignes en double pour garantir des entrées uniques.

In [2]:
csv_userfilm_file = "/Users/valid/Desktop/sda-proj/spotusers.csv"
userfilm_data = pd.read_csv(csv_userfilm_file, header=None, names=["user_id", "film_id"], nrows=500000)
userfilm_data = userfilm_data.drop_duplicates()


Transformation et création de structures de données utiles pour l'analyse :
- Grouper les films par utilisateur pour créer un dictionnaire où chaque utilisateur est associé à une liste de films qu'il a regardés.
- Extraire tous les films uniques du dataset et les trier pour garantir un ordre cohérent.
- Créer un dictionnaire qui associe chaque film unique à un index numérique, utile pour des opérations comme la création de matrices ou le traitement avec des algorithmes.

In [3]:
user_to_films = userfilm_data.groupby("user_id")["film_id"].apply(list).to_dict()
unique_films = sorted(set(userfilm_data["film_id"]))
film_to_index = {film: idx for idx, film in enumerate(unique_films)}

Cette étape permet de réduire la consommation de mémoire, car les identifiants des utilisateurs sont stockés sous forme d'indices numériques, plutôt que sous forme de chaînes de caractères.

In [ ]:
unique_users = sorted(user_to_films.keys())
user_to_index = {user: idx for idx, user in enumerate(unique_users)}

Définition de la taille du vecteur et initialisation de l'index Annoy :
- La taille du vecteur est définie en fonction du nombre de films uniques, car chaque film sera représenté par un vecteur dans l'index.
- Initialisation de l'index Annoy en spécifiant la taille du vecteur et en utilisant la métrique 'angular' (qui est souvent utilisée pour des données de type vecteur d'angle).

In [ ]:
vector_size = len(unique_films)
ann = AnnoyIndex(vector_size, metric='angular')

Cette étape consiste à créer et ajouter les vecteurs utilisateur à l'index Annoy, tout en utilisant les indices numériques pour les utilisateurs afin de réduire la consommation de mémoire.
- Pour chaque utilisateur, on récupère son index numérique via le dictionnaire user_to_index.
- On initialise un vecteur de zéros représentant les films que l'utilisateur n'a pas vus.
- Pour chaque film vu par l'utilisateur, la position correspondante dans le vecteur est mise à 1.
- Enfin, le vecteur est ajouté à l'index Annoy en utilisant l'index de l'utilisateur.

In [ ]:
for user_id, films in user_to_films.items():
    user_vector = np.zeros(vector_size)
    for film in films:
        user_vector[film_to_index[film]] = 1
    ann.add_item(user_id, user_vector)

La méthode ann.build(7) construit l'index avec 7 arbres.
- Plus le nombre d'arbres est élevé, plus l'index est précis, mais cela consomme plus de mémoire et de temps.
- 7 arbres offrent un bon compromis entre précision et performance pour des données de taille moyenne à grande.

In [ ]:
ann.build(7)
ann.save("C:/Users/valid/Desktop/sda-proj/sda-proj.ann")

La fonction recommend_for_user recommande des films à un utilisateur en se basant sur les utilisateurs similaires, en utilisant l'index Annoy pour la recherche de voisins.
- Si l'utilisateur n'est pas trouvé, un message d'erreur est renvoyé.
- Ensuite, l'index des utilisateurs similaires est récupéré, en excluant l'utilisateur lui-même.
- Les films regardés par ces utilisateurs similaires sont collectés, puis les films déjà vus par l'utilisateur d'origine sont supprimés de la liste des recommandations.
- La fonction retourne une liste des films recommandés.

In [ ]:
def recommend_for_user(user_id, n_recommendations=5):
    if user_id not in user_to_index:
        return "User not found in dataset."
    
    user_index = user_to_index[user_id]
    similar_users = ann.get_nns_by_item(user_index, n_recommendations + 1)
    similar_users = [unique_users[idx] for idx in similar_users if idx != user_index]

    recommended_films = set()
    for similar_user in similar_users:
        recommended_films.update(user_to_films[similar_user])

    watched_films = set(user_to_films[user_id])
    recommended_films.difference_update(watched_films)

    return list(recommended_films)

C'est un exemple d'utilisation de la fonction recommend_for_user.

In [ ]:
u_id = 663821
recommendations = recommend_for_user(u_id)
print(f"Recommendations for user {u_id}: {recommendations}")

---
Lien vers le github repo: https://github.com/DjPetuhe/sda-proj